<a href="https://colab.research.google.com/github/borislevant/CourseraMachineLearning/blob/master/cifar10_from_softmax_to_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Introduction.
The CIFAR-10 dataset contains 60,000 color images of 32 x 32 pixels in 3 channels divided into 10
classes. Each class contains 6,000 images. The training set contains 50,000 images, while the test sets
provides 10,000 images. This image taken from the CIFAR repository ( <a href = "https://www.cs.toronto.edu/~kriz/cifar.html">https://www.cs.toronto.edu/~kriz/cifar.html </a>). This is a classification problem with 10 classes(muti-label classification). We can take a view on this image for more comprehension of the dataset. 

![cifar10.png](https://github.com/borislevant/SciComPy/blob/master/cifar10.png?raw=1)


The challenge is to recognize previously unseen images and assign them to one of the 10 classes.

Ok Let's get started.

## 2. Import and Preprocess the data

### 2.1 Import all required libraries

In [2]:
from __future__ import print_function
from tensorflow import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, InputLayer
from keras.layers import Conv2D, MaxPooling2D
import os

import numpy as np

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import itertools

%matplotlib inline


In [3]:
batch_size = 32  # The default batch size of keras.
num_classes = 10  # Number of class for the dataset
epochs = 20
data_augmentation = False

###2.2 Import and preproces of data 
We load the data and split it between train and test sets


In [4]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [5]:
# Normalize the data. Before we need to connvert data type to float for computation.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices. This is called one hot encoding.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

##3.Softmax model

In [6]:
model = Sequential()
model.add(InputLayer(input_shape=x_train.shape[1:]))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 10)                30730     
                                                                 
Total params: 30,730
Trainable params: 30,730
Non-trainable params: 0
_________________________________________________________________


In [7]:
# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [8]:
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)


Epoch 1/20
1563/1563 [==============================] - 6s 3ms/step - loss: 1.9746 - accuracy: 0.2970 - val_loss: 1.8889 - val_accuracy: 0.3337
Epoch 2/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.8503 - accuracy: 0.3508 - val_loss: 1.8417 - val_accuracy: 0.3572
Epoch 3/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.8148 - accuracy: 0.3687 - val_loss: 1.8176 - val_accuracy: 0.3616
Epoch 4/20
1563/1563 [==============================] - 4s 3ms/step - loss: 1.7952 - accuracy: 0.3763 - val_loss: 1.8013 - val_accuracy: 0.3616
Epoch 5/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7808 - accuracy: 0.3840 - val_loss: 1.7969 - val_accuracy: 0.3728
Epoch 6/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7700 - accuracy: 0.3872 - val_loss: 1.8074 - val_accuracy: 0.3748
Epoch 7/20
1563/1563 [==============================] - 5s 3ms/step - loss: 1.7611 - accuracy: 0.3883 - val_loss: 1.7695 - val_accuracy:

##3.Fully-connected Neural Network (FNN) model

In [9]:
model = Sequential()
model.add(InputLayer(input_shape=x_train.shape[1:]))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 1024)              3146752   
                                                                 
 dense_2 (Dense)             (None, 256)               262400    
                                                                 
 dense_3 (Dense)             (None, 64)                16448     
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
Total params: 3,426,250
Trainable params: 3,426,250
Non-trainable params: 0
_________________________________________________________________


In [10]:
# initiate RMSprop optimizer
opt = keras.optimizers.Adam(learning_rate=0.001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)

Epoch 1/20
1563/1563 [==============================] - 8s 5ms/step - loss: 1.9099 - accuracy: 0.3023 - val_loss: 1.7831 - val_accuracy: 0.3557
Epoch 2/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.6960 - accuracy: 0.3876 - val_loss: 1.6710 - val_accuracy: 0.3992
Epoch 3/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.6043 - accuracy: 0.4249 - val_loss: 1.5615 - val_accuracy: 0.4422
Epoch 4/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5467 - accuracy: 0.4460 - val_loss: 1.5434 - val_accuracy: 0.4506
Epoch 5/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5038 - accuracy: 0.4611 - val_loss: 1.5122 - val_accuracy: 0.4602
Epoch 6/20
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4680 - accuracy: 0.4737 - val_loss: 1.5242 - val_accuracy: 0.4541
Epoch 7/20
1563/1563 [==============================] - 7s 4ms/step - loss: 1.4441 - accuracy: 0.4832 - val_loss: 1.5040 - val_accuracy: